### Load dependencies 
We start by importing all necessary python packages, and the functionalities implemented in the utils folder of this repo.

In [4]:
from utils.inference_YOLO import *
from utils.model_eval import *
from sahi.predict import predict

### Insert paths, output directories, patch dimensions etc.
Here we specify the inference parameters, including which model and test set to use, the dimensions of the patches, their overlap, etc. Some general info:
- We use 640x640 patches in the paper. 
- The amount of overlap can be calculated from the values specified in Table 2. E.g.: $\frac{128}{640} = 0.2$
- The DoR and IoU values can be found in Table 12.
- The radii can be found in the Size Avg. column of Table 1. They need to be passed as a dictionary where `key = class ID`, `value = radius`. The ID of a species is its position in the Species column of Table 1. E.g., for the JE-TL19 dataset, the `radii` dict would look as follows: `{0: 62, 1: 81, 2: 49}`
- `classID2name` is a dictionary mapping class ids to their name. Again, for the JE-TL19 dataset: `classID2name = {0: "Elephant", 1: "Giraffe", 2: "Zebra"}`

In [ ]:
imgs_dir = ""   #TODO: Insert Path to image directory (as downloaded from Zenodo).
output_dir = ""     #TODO: Insert path to directory you want your output stored in
patch_dims = {"width": -1, "height": -1}  #TODO: Insert patch width and height; (640, 640) in the paper
ovrlp = -1.0    #TODO: Define patch overlap as a fraction of the patch dimensions (e.g. 128 pixel overlap = 0.2 of 640x640 patches; cf. Table 2)
dor_thresh = -1.0    #TODO: Define DoR threshold for NMS and computing evaluation metrics (cf. Table 12)
iou_thresh = -1.0    #TODO: Define IoU threshold for NMS and computing evaluation metrics (cf. Table 12)
mdl_path = ""     #TODO: Insert path to the model .pt file
radii = {0: None, 1: None, 2: None}     #TODO: Insert radius for each species in the dataset; cf. Table 1.
classID2name = {0: "", 1: "", 2: ""}    #TODO: Insert class ids and names as listed in Table 1.
img_format = ""     #TODO: Specify the image format (suffix). E.g.: "JPG"/"jpg" (case sensitive)
device = ""     #TODO: Insert device to be used for inference. "cuda" if you have access to a GPU, "cpu" otherwise.

### Set some more paths (no input required)
Here the path to the file containing the test set annotations, and to the tiling folder are set. The annotations file is needed to compute evaluation metrics and counting errors after inference, and the tiling folder is where the patches extracted from each image are going to be stored. We also set the random seed for reproducibility. 

In [ ]:
ann_file = f"{imgs_dir}/test_annotations.json"
tiling_dir = f"{imgs_dir}/tiles"
random.seed(0)

### Define Task
Here we specify what model we will be using. Set the `task` variable to `"locate"` if you are working with a POLO model, use `"detect"` otherwise.

In [ ]:
task = "locate"

### Run tiled inference (no input required)
This is where we run the actual inference For bounding box models, we use the `SAHI` library, for POLO we use the methods implemented in `utils/inference_POLO.py`. `coco_file_path` will point to a json file required by `SAHI` to run tiled inference for bounding box models.

In [ ]:
if task == "locate":
    run_tiled_inference_POLO(model=mdl_path, 
                             class_ids=list(radii.keys()),
                             imgs_dir=imgs_dir, 
                             img_files_ext=img_format,
                             patch_dims=patch_dims, 
                             patch_overlap=ovrlp, 
                             output_dir=output_dir,
                             dor_thresh=dor_thresh,
                             radii=radii,
                             ann_file=ann_file,
                             ann_format="PT_DEFAULT")
else:
    coco_file_path = [p for p in Path(imgs_dir).glob("*.json") if "coco" in p][0]
    predict(
        model_type="yolov8",
        model_path=mdl_path,
        model_device=device, 
        source=imgs_dir,
        slice_height=patch_dims["height"],
        slice_width=patch_dims["width"],
        overlap_height_ratio=ovrlp,
        overlap_width_ratio=ovrlp,
        postprocess_match_threshold=iou_thresh,
        dataset_json_path=coco_file_path,
        project=output_dir, 
        name="output_SAHI",
        novisual=True, 
        verbose=0
    )


### Compute Evaluation metrics
This cell evaluated the ouputs of the previous cell. It will generate a number of files:
- `count_diffs_img_lvl.xlsx`: Excel sheet containing the difference between predicted and ground truth count for each image.
- `counts_gt_pred_*.png`: Plot of predicted vs. forund truth count for class `*`.
- `counts_total.json`: Predicted counts summed over all images.
- `em.json`: Evaluation metrics.
- `errors_img_lvl.json`: Counting metrics.
- `F1_curve.png`: F1 score plotted against the confidence threshold.
- `P_curve.png`: Precision plotted against the confidence threshold.
- `R_curve.png`: Recall plotted against the confidence threshold.

Before running the cell, please set the `is_pseudo` variable to `True` if you are using a `YOLOv8` model trained on pseudo boxes.

In [ ]:
if task == "detect":
    is_pseudo = None    #TODO: Set to True if you are using a YOLOv8 model trained on pseudo boxes

    box_dims = {cid: {"width": radii[cid], "height": radii[cid]} for cid in radii.keys()} if is_pseudo else None
    read_output_SAHI(out_json_SAHI=f"{output_dir}/output_SAHI/result.json", dataset_json_SAHI=coco_file_path, class_ids=list(classID2name.keys()), 
                         iou_thresh=iou_thresh, box_dims=box_dims, ann_file=ann_file, ann_format="BX_WH", output_dir=output_dir)

compute_errors_img_lvl(gt_counts_dir=f"{imgs_dir}/image_counts", pred_counts_dir=f"{output_dir}/detections", class_ids=list(classID2name.keys()), 
                           output_dir=output_dir)
compute_em_img_lvl(preds_dir=f"{output_dir}/detections", class_id2name=classID2name, task=task, output_dir=output_dir)    